# 1. Load notes and structured data tables

In [1]:
import sys; print(sys.executable); print(sys.path)

""" NOTE TO SELF: (todo: look into this later, python path mismatch in openotes venv)
Source: https://github.com/jupyter/notebook/issues/2359#issuecomment-318056024

If having installation issues, try

[path to python] -m pip install [package]

where [path to python] is the output of sys.executable.

e.g.
/opt/conda/envs/opennotes/bin/python -m pip install scispacy==0.2.5 spacy==2.3.2 nltk

Issues with spacy & scispacy versions: these specific versions should work

https://stackoverflow.com/questions/66149878/e053-could-not-read-config-cfg-resumeparser
# !pip install spacy==2.3.2 or 2.3.5
# !pip install scispacy==0.2.5
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz

# loading with spacy==2.3.5 works
>>> import spacy
>>> spacy.__version__
'2.3.5'
>>> sci_nlp = spacy.load("en_core_sci_sm")
>>> sci_nlp
<spacy.lang.en.English object at 0x7f3c82ec3e90>
"""

/opt/conda/envs/opennotes/bin/python
['/home/yutsumi/opennotes-error-detection', '/opt/conda/envs/opennotes/lib/python37.zip', '/opt/conda/envs/opennotes/lib/python3.7', '/opt/conda/envs/opennotes/lib/python3.7/lib-dynload', '', '/opt/conda/envs/opennotes/lib/python3.7/site-packages', '/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/extensions', '/home/yutsumi/.ipython']


' NOTE TO SELF: (todo: look into this later, python path mismatch in openotes venv)\nSource: https://github.com/jupyter/notebook/issues/2359#issuecomment-318056024\n\nIf having installation issues, try\n\n[path to python] -m pip install [package]\n\nwhere [path to python] is the output of sys.executable.\n\ne.g.\n/opt/conda/envs/opennotes/bin/python -m pip install scispacy==0.2.5 spacy==2.3.2 nltk\n\nIssues with spacy & scispacy versions: these specific versions should work\n\nhttps://stackoverflow.com/questions/66149878/e053-could-not-read-config-cfg-resumeparser\n# !pip install spacy==2.3.2 or 2.3.5\n# !pip install scispacy==0.2.5\n# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz\n\n# loading with spacy==2.3.5 works\n>>> import spacy\n>>> spacy.__version__\n\'2.3.5\'\n>>> sci_nlp = spacy.load("en_core_sci_sm")\n>>> sci_nlp\n<spacy.lang.en.English object at 0x7f3c82ec3e90>\n'

In [2]:
# Import libraries
import os 
import random
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime
from collections import Counter

In [3]:
# Load MIMIC tables
notes_df  = pd.read_csv('NOTEEVENTS.csv.gz',    compression='gzip', error_bad_lines=False)
drug_df   = pd.read_csv('PRESCRIPTIONS.csv.gz', compression='gzip', error_bad_lines=False)
lab_df    = pd.read_csv('LABEVENTS.csv.gz',     compression='gzip', error_bad_lines=False)
d_lab_df  = pd.read_csv('D_LABITEMS.csv.gz',    compression='gzip', error_bad_lines=False)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
class Patient(object):
    def __init__(self, hadm_id, notes_df, drug_df, lab_df, d_lab_df):
        self.hadm_id = hadm_id
        
        # this patient's data
        self.notes_df = self.filter_duplicates(notes_df.loc[notes_df['HADM_ID'] == hadm_id])
        self.drug_df  = drug_df.loc[drug_df['HADM_ID'] == hadm_id]
        self.lab_df   = lab_df.loc[lab_df['HADM_ID'] == hadm_id]
        
        self.d_lab_df = d_lab_df # lab ditems df
        
    def filter_duplicates(self, pat_notes_df):
        # Filtering out duplicate / autosave's -- only take the longest
        for cat in pat_notes_df.CATEGORY.unique(): 
            cat_notes_df = pat_notes_df.loc[pat_notes_df.CATEGORY == cat]
            for time in cat_notes_df.CHARTTIME.unique():
                time_notes_df = cat_notes_df.loc[cat_notes_df.CHARTTIME == time]
                if len(time_notes_df) > 1:
                    # get indices of first N-1 shortest rows
                    idx_to_drop = time_notes_df.TEXT.apply(lambda x: len(x)).sort_index().index[:-1]
                    pat_notes_df = pat_notes_df.drop(idx_to_drop) # drop by row index
                    
        return pat_notes_df
    
class Row(object):
    def __init__(self, patient):
        pass
    
    @property
    def hadm_id(self):
        return self.patient.hadm_id

class Note(Row):
    def __init__(self, patient, row_id):
        self.patient = patient                                                   # patient this note is for
        self.df      = patient.notes_df.loc[patient.notes_df.ROW_ID == row_id]   # df row for this note
        self.txt     = self.df.TEXT.item()                                       # note in string format
        self.cat     = self.df.CATEGORY.item()                                   # note category
        
        # Get datetime
        if type(self.df.CHARTTIME) == str:
            self.time = datetime.strptime(self.df.CHARTTIME, "%Y-%m-%d %H:%M:%S")
        elif type(self.df.CHARTDATE) == str:
            self.time = datetime.strptime(self.df.CHARTDATE, "%Y-%m-%d")
        else:
            self.time = None
            
        # Tokenize note
        
class Drugs(Row):
    def __init__(self, patient):
        pass

class Labs(Row):
    def __init__(self, patient):
        pass
    
class Sentence(object):
    def __init__(self, note, sentence_idx):
        pass

# class Sentence(object):
#     def __init__(self, sentence, sci_nlp, linker, med7_nlp, filter_map=None, conflict_map=None):
#         """ 
#         Given a sentence and scispacy model with UMLS linker and Med7 model,
#         SentRep extracts important information and stores them as attributes. 
        
#         Set filter_map to be map of semantic type identifiers to name (e.g. SEMANTIC_TYPE_TO_NAME)
#         to only extract those attributes.
#         """
#         self.umls_cui_map = linker.umls.cui_to_entity # maps CUI to entity information
#         self.filter_map   = filter_map
#         self.conflict_map = conflict_map
#         self.is_filter    = (filter_map is not None)
#         self.is_conflict  = (conflict_map is not None)
        
#         self.sci_doc = sci_nlp(sentence)
#         self.med_doc = med7_nlp(sentence)
        
#         self.semantic_types = []
#         self.semantic_names = []  # names of categories of entities
#         self.canonical_names = [] # names of types of entities
#         self.get_umls_info()

#     def get_umls_info(self):
#         for ent in self.sci_doc.ents: # extract info (umls) for each entity
#             # todo: look into this bug, ent._.umls_ents sometimes empty list
#             try:
#                 cui, _ = ent._.umls_ents[0] # assuming `max_entites_per_mention=1` for now
#             except IndexError:
#                 continue
#             cui_info = self.umls_cui_map[cui]
            
#             ent_valid_type_list = [t in self.filter_map for t in cui_info.types]
#             ent_valid_type = any(ent_valid_type_list) # checks if entity is a valid type
            
#             if not self.is_filter or ent_valid_type: # only add to list if we're not filtering of it's valid
#                 self.canonical_names.append(cui_info.canonical_name)
#                 for (stype, keep) in zip(cui_info.types, ent_valid_type_list):
#                     if keep:
#                         self.semantic_types.append(stype)
#                         self.semantic_names.append(self.filter_map[stype])
            
#         self.semantic_types = set(self.semantic_types)
#         self.semantic_names = set(self.semantic_names)
#         self.canonical_names = set(self.canonical_names)
        
#     def similarity(self, srep):
#         """ Given another SentenceRep instance, compares similarity.
        
#         e.g.
#         srep.similarity(srep)   # measuring similarity with itself
#         >> 1.0                  # 1.0 is maximum score
#         """
#         return self.sci_doc.similarity(srep.doc)
    
#     def is_ctype(self, ctype):
#         """ Given a conflict type (e.g. "diagnosis"),
#             returns True if this sentence falls into that category, False otherwise.
#             Returns None if conflict_map is undefined.
#         """
#         if self.is_conflict: 
#             ctype_stypes = self.conflict_map[ctype] # get list of semantic types for this conflict
#             return any([stype in ctype_stypes for stype in self.semantic_types])
#         return None

In [40]:
pat = Patient(115637, notes_df, drug_df, lab_df, d_lab_df)
note = Note(pat, 32296)

note.hadm_id
note.txt
note.cat
note.time

In [26]:
note = pat.notes_df.loc[pat.notes_df.ROW_ID == 32296]

note.CATEGORY.item()
# note.CHARTDATE
# note.CHARTTIME
# note.TEXT

'Discharge summary'

In [ ]:
# Get notes / sentences for patient
def get_tokenized_sentences(notes_df, hadm_id, filter_labs=True, filter_titles=True):
  result_dict = {}

  subset = notes_df.loc[notes_df["HADM_ID"] == hadm_id]

  # Filtering out duplicate / autosave's -- only take the longest
  for cat in subset.CATEGORY.unique(): 
    cat_subset = subset.loc[subset.CATEGORY == cat]
    for time in cat_subset.CHARTTIME.unique():
      cat_time_subset = cat_subset.loc[cat_subset.CHARTTIME == time]
      if len(cat_time_subset) > 1:
        # get indices of first N-1 shortest rows
        idx_to_drop = cat_time_subset.TEXT.apply(lambda x: len(x)).sort_index().index[:-1]
        subset = subset.drop(idx_to_drop) # drop by row index
        
  # for each type of note / daily note for hadm_id
  for i in tqdm(range(len(subset))):
    row = subset.iloc[i]
    note = row["TEXT"]
    row_id = row["ROW_ID"]
    sents = !python mimic-tokenize/heuristic-tokenize.py "{note}"
    ind_sentences = sents[0].split(", \'")
    
    if filter_labs:
        ind_sentences = delete_copied_lab_tables(ind_sentences)
    if filter_titles:
        ind_sentences = remove_titles(ind_sentences)
#     result_dict[row_id] = new_sentences

    # Get time
    if type(row.CHARTTIME)==str:
        time = datetime.strptime(row.CHARTTIME, "%Y-%m-%d %H:%M:%S")
    elif type(row.CHARTDATE)==str:
        time = datetime.strptime(row.CHARTDATE, "%Y-%m-%d")
    else:
        time = None
        
    result_dict[row_id] = (ind_sentences, time, row.CATEGORY)
  
  # result_dict key is the row id in note_df
  # result_dict value is the list of tokenized sentences
  return result_dict

# for Yuria: make sure to include this cell in the pipeline

import re

def diff_list(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

def delete_copied_lab_tables(ind_sentences):

  # [**yyyy-mm-dd**], 02:10
#   rgx_list = ["[\*\*\d{4}\-\d{1,2}\-\d{1,2}\*\*]", "\d{1,2}\-\d{1,2}"]
#   rgx_list = ["[\*\*[0-9]{4}-[0-9]{1,2}-[0-9]{1,2}\*\*] *[0-9]{1,2}-[0-9]{1,2}"]
#   rgx_list = ["[\*\*[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]\*\*]   [0-9][0-9]-[0-9][0-9]"]
  rgx_list = ["[\*\*[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]\*\*]"]
#   rgx_list = ["[\d{4}\-\d{1,2}\-\d{1,2}][^\S]+\d{1,2}\-\d{1,2}"]

  delete_list = []
  # ind_sentences is list of strings
  for sentence in ind_sentences:
    for rgx_match in rgx_list:
      match = re.search(rgx_match, sentence)
      if match and sentence not in delete_list:
        delete_list.append(sentence)
#         print(f"Sentence: {sentence}")
#         print("Deleted")

  return diff_list(ind_sentences, delete_list)

def remove_titles(sentences):
    """ Omits anything that has ':' in last two entries of the string. 
    e.g. "...Results:"
    """
    return list(filter(lambda x: ':' not in x[-2:], sentences))

In [4]:
# Get notes / sentences for patient
def get_tokenized_sentences(notes_df, hadm_id, filter_labs=True, filter_titles=True):
  result_dict = {}

  subset = notes_df.loc[notes_df["HADM_ID"] == hadm_id]

  # Filtering out duplicate / autosave's -- only take the longest
  for cat in subset.CATEGORY.unique(): 
    cat_subset = subset.loc[subset.CATEGORY == cat]
    for time in cat_subset.CHARTTIME.unique():
      cat_time_subset = cat_subset.loc[cat_subset.CHARTTIME == time]
      if len(cat_time_subset) > 1:
        # get indices of first N-1 shortest rows
        idx_to_drop = cat_time_subset.TEXT.apply(lambda x: len(x)).sort_index().index[:-1]
        subset = subset.drop(idx_to_drop) # drop by row index
        
  # for each type of note / daily note for hadm_id
  for i in tqdm(range(len(subset))):
    row = subset.iloc[i]
    note = row["TEXT"]
    row_id = row["ROW_ID"]
    sents = !python mimic-tokenize/heuristic-tokenize.py "{note}"
    ind_sentences = sents[0].split(", \'")
    
    if filter_labs:
        ind_sentences = delete_copied_lab_tables(ind_sentences)
    if filter_titles:
        ind_sentences = remove_titles(ind_sentences)
#     result_dict[row_id] = new_sentences

    # Get time
    if type(row.CHARTTIME)==str:
        time = datetime.strptime(row.CHARTTIME, "%Y-%m-%d %H:%M:%S")
    elif type(row.CHARTDATE)==str:
        time = datetime.strptime(row.CHARTDATE, "%Y-%m-%d")
    else:
        time = None
        
    result_dict[row_id] = (ind_sentences, time, row.CATEGORY)
  
  # result_dict key is the row id in note_df
  # result_dict value is the list of tokenized sentences
  return result_dict

# for Yuria: make sure to include this cell in the pipeline

import re

def diff_list(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

def delete_copied_lab_tables(ind_sentences):

  # [**yyyy-mm-dd**], 02:10
#   rgx_list = ["[\*\*\d{4}\-\d{1,2}\-\d{1,2}\*\*]", "\d{1,2}\-\d{1,2}"]
#   rgx_list = ["[\*\*[0-9]{4}-[0-9]{1,2}-[0-9]{1,2}\*\*] *[0-9]{1,2}-[0-9]{1,2}"]
#   rgx_list = ["[\*\*[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]\*\*]   [0-9][0-9]-[0-9][0-9]"]
  rgx_list = ["[\*\*[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]\*\*]"]
#   rgx_list = ["[\d{4}\-\d{1,2}\-\d{1,2}][^\S]+\d{1,2}\-\d{1,2}"]

  delete_list = []
  # ind_sentences is list of strings
  for sentence in ind_sentences:
    for rgx_match in rgx_list:
      match = re.search(rgx_match, sentence)
      if match and sentence not in delete_list:
        delete_list.append(sentence)
#         print(f"Sentence: {sentence}")
#         print("Deleted")

  return diff_list(ind_sentences, delete_list)

def remove_titles(sentences):
    """ Omits anything that has ':' in last two entries of the string. 
    e.g. "...Results:"
    """
    return list(filter(lambda x: ':' not in x[-2:], sentences))

Given hadm_id, convert all its prescriptions to sentences

In [12]:
def scripts_to_sentence(id, script_df):
  """
  id: HADM_ID for patient
  script_df: data frame of prescription table

  returns: dictionary: day -> set of sentences describing prescriptions given that day
  """
  template = "Patient given {}."
  # for each patient admission, create map of drug to its dosage info
  daily_drugs = {}
  # get patient specific data
  patient_data = script_df.loc[script_df['HADM_ID']==id]
  unique_drugs = patient_data.DRUG.unique()
  # record info for each unique drug the patient is on
  for drug in unique_drugs:
    drug_data = patient_data.loc[patient_data['DRUG']==drug]
    for i, row in drug_data.iterrows():
      starttime = row['STARTDATE']
      startday = starttime.split()[0]
      if startday not in daily_drugs:
        daily_drugs[startday] = set()
      daily_drugs[startday].add(template.format(drug.lower()))
  return daily_drugs

/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
daily_drugs = scripts_to_sentence(hadm_id, script_df)
daily_drugs

{'2157-05-16': {'Patient given 0.9% sodium chloride.',
  'Patient given 1/2 ns.',
  'Patient given acetaminophen.',
  'Patient given atenolol.',
  'Patient given atorvastatin.',
  'Patient given ceftriaxone.',
  'Patient given cepacol (menthol).',
  'Patient given ciprofloxacin hcl.',
  'Patient given docusate sodium.',
  'Patient given glipizide.',
  'Patient given heparin.',
  'Patient given hyoscyamine.',
  'Patient given insulin.',
  'Patient given iso-osmotic dextrose.',
  'Patient given lr.',
  'Patient given magnesium sulfate.',
  'Patient given metronidazole (flagyl).',
  'Patient given ns.',
  'Patient given ondansetron.',
  'Patient given phenazopyridine.',
  'Patient given sodium chloride 0.9%  flush.',
  'Patient given sw.',
  'Patient given vancomycin.',
  'Patient given zolpidem tartrate.'},
 '2157-05-18': {'Patient given 0.9% sodium chloride.',
  'Patient given iso-osmotic dextrose.',
  'Patient given lidocaine 1%.',
  'Patient given micafungin.',
  'Patient given sodium

Given hadm_id, convert all labs run to sentences.

In [22]:
# given item id, find the corresponding label
def find_label(item, d_lab_df):
  l = d_lab_df.loc[d_lab_df["ITEMID"] == item]
  for i, row in l.iterrows():
    return row["LABEL"]
  return None

def labs_to_sentence(id, lab_df, d_lab_df):
  """
  id: HADM_ID for patient
  lab_df: data frame of lab events table
  d_lab_df: data frame of lab events item ID table

  returns: dictionary: day -> set of sentences describing labs for that day
  """
  normal_template = "{} lab came back normal."
  abnormal_template = "{} lab came back abnormal."
  id_data = lab_df.loc[lab_df["HADM_ID"] == id]
  daily_labs = {}
  for i, row in id_data.iterrows():
    item = row["ITEMID"]
    day = row["CHARTTIME"].split()[0]
    if day not in daily_labs:
      daily_labs[day] = set()
    label = find_label(item, d_lab_df)
    if label is None:
      continue
    if row["FLAG"] == "abnormal":
      daily_labs[day].add(abnormal_template.format(label))
    else:
      daily_labs[day].add(normal_template.format(label))
  return daily_labs

In [24]:
labs_to_sentence(hadm_id, lab_df, d_lab_df)

{'2157-05-16': {'Alanine Aminotransferase (ALT) lab came back abnormal.',
  'Albumin lab came back normal.',
  'Alkaline Phosphatase lab came back normal.',
  'Amylase lab came back normal.',
  'Anion Gap lab came back normal.',
  'Anisocytosis lab came back normal.',
  'Asparate Aminotransferase (AST) lab came back normal.',
  'Atypical Lymphocytes lab came back abnormal.',
  'Bacteria lab came back normal.',
  'Bands lab came back abnormal.',
  'Basophils lab came back normal.',
  'Bicarbonate lab came back normal.',
  'Bilirubin lab came back normal.',
  'Bilirubin, Total lab came back normal.',
  'Blood lab came back normal.',
  'Burr Cells lab came back normal.',
  'Calcium, Total lab came back normal.',
  'Chloride lab came back normal.',
  'Creatinine lab came back abnormal.',
  'Creatinine, Urine lab came back normal.',
  'EDTA Hold lab came back normal.',
  'Eosinophils lab came back normal.',
  'Epithelial Cells lab came back normal.',
  'Glucose lab came back abnormal.',
  '

In [5]:
data.HADM_ID.values

array([167853., 107527., 167118., ..., 115637., 115637., 115637.])

In [8]:
# Load HADM ID's with consecutive physician notes
if os.path.exists("hadm_ids.pkl"):
    with open("hadm_ids.pkl", "rb") as f:
        hadm_ids = pickle.load(f)
else:
    hadm_ids = []
    for hadm_id in tqdm(data.HADM_ID.unique()):
        hadm_data = data.loc[data.HADM_ID == hadm_id]
        hadm_phys_notes = hadm_data.loc[hadm_data.CATEGORY == "Physician "]

        if len(hadm_phys_notes) > 1:
            hadm_ids.append(hadm_id)

    with open("hadm_ids.pkl", "wb") as f:
        pickle.dump(hadm_ids, f)

In [9]:
len(hadm_ids)

8733

README: You can set to different HADM ID's here. `hadm_ids` contains all id's of patients with consecutive physician notes.

In [11]:
# hadm_id = hadm_ids[46]
hadm_id = 171833
print(f"HADM ID={hadm_id}")

hdata = data.loc[data.HADM_ID == hadm_id]
# print(hdata.loc[hdata.CATEGORY == "Discharge summary"].TEXT.values[0])

# get dict of list of sentences for each note
sents_dict = get_tokenized_sentences(data, hadm_id, filter_labs=True,  filter_titles=True)

print(list(map(lambda x: x[2], sents_dict.values())))

# get list of sentences
all_sent_strs  = []  # sentences
all_sent_times = []  # corresponding times
for row_id, (row_sents, row_time, row_category) in sents_dict.items():
#     if 'physician' in row_category.lower() or 'nursing' in row_category.lower():
    if 'physician' in row_category.lower():
        all_sent_strs.extend(row_sents)
        all_sent_times.extend([row_time for _ in range(len(row_sents))])
        
print(all_sent_strs[:10])
print(len(all_sent_strs))

  0%|          | 0/19 [00:00<?, ?it/s]

HADM ID=171833


100%|██████████| 19/19 [00:22<00:00,  1.19s/it]

['Discharge summary', 'Nursing', 'Physician ', 'Physician ', 'Respiratory ', 'Physician ', 'Physician ', 'Nursing', 'Physician ', 'Nursing', 'Nursing', 'Nursing', 'Physician ', 'Radiology', 'Radiology', 'Radiology', 'Radiology', 'Radiology', 'Radiology']
["HPI:    Pt with recent admit with abdominal pain and found to have    diveritculitis and nephrolithitis.'", "   Skin:  Not assessed'", "4.1    138'", 'Patient admitted from:  [**Hospital1 1**]  OR / PACU    History obtained from  [**Hospital 31**]  Medical records    Allergies:    Aspirin    Unknown;    Motrin (Oral) (Ibuprofen)    Unknown;    Trilisate (Oral) (Chol  [**Doctor First Name **] /Magnesium Salicylate)    Unknown;    Naprosyn (Oral) (Naproxen)    Unknown;    Toradol (Injection) (Ketorolac Tromethamine)    Unknown;    Vicodin (Oral) (Hydrocodone Bit/Acetaminophen)    Unknown;    Percocet (Oral) (Oxycodone Hcl/Acetaminophen)    Unknown;    Indocin (Oral) (Indomethacin)    Unknown;    Dilaudid (Injection) (Hydromorphone Hcl/

# 2. Extracting concepts from sentences

Some options were [MetaMap](https://metamap.nlm.nih.gov/) and [spaCy](https://spacy.io/). 

[MetaMap](https://metamap.nlm.nih.gov/) is specific to recognizing UMLS concepts. There is a [Python wrapper](https://github.com/AnthonyMRios/pymetamap), but known to be slow and bad.

[spaCy](https://spacy.io/) is a popular NLP Python package with an extensive library for named entity recognition. It has a wide variety of [extensions](https://spacy.io/universe) and models to choose from. We're going with the following.

* [scispaCy](https://spacy.io/universe/project/scispacy) contains spaCy models for processing biomedical, scientific or clinical text. It seems easy to use and has a wide variety of concepts it can recognize, including UMLS, RxNorm, etc.

* [negspaCy](https://spacy.io/universe/project/negspacy) identifies negations using some extension of regEx. Probably useful for things like, "this pt is diabetic" v. "this pt is not diabetic." [todo: negation identification of medspacy might be better, https://github.com/medspacy/medspacy]

* [Med7](https://github.com/kormilitzin/med7) is a model trained for recognizing entities in prescription text, e.g. identifies drug name, dosage, duration, etc., which could be useful stuff to check for conflicts. 

We're going with spaCy for this.. and coming up with a coherent way to integrate entities picked up by these three extensions/models.

!python --version

## i) Installations

In [19]:
sys.executable

'/opt/conda/envs/opennotes/bin/python'

In [20]:
# Should be installed in conda env but run if you run into error
# If you still run into error, run:
# `![whatever `sys.executable` prints out] -m pip install [package]`
# !/opt/conda/envs/opennotes/bin/python -m pip install spacy==2.3.2
# !/opt/conda/envs/opennotes/bin/python -m pip install scispacy==0.2.5
# !/opt/conda/envs/opennotes/bin/python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz

In [21]:
import spacy
import scispacy

from pprint import pprint
from collections import OrderedDict

from spacy import displacy
# from scispacy.abbreviation import AbbreviationDetector # UMLS already contains abbrev. detect
from scispacy.umls_linking import UmlsEntityLinker

In [22]:
spacy.__version__

'2.3.5'

## ii) Setting up the model

The model is used to form word/sentence embeddings for the NER task. Thus, it's important to choose model that has been tuned for our specific use case (e.g. clinical text, prescription information) so the embeddings are useful for naming the entity.

[Note to self:] one potential idea to look into if we have time remaining, something about using custom model for spacy pipeline (could we do smth with the romanov models since they've been trained specifically for conflict detection?) -- https://spacy.io/usage/v3

### a) scispaCy

For scispaCy, we set up one of their models that has been trained on biomedical data. Other models can be found [here](https://allenai.github.io/scispacy/). 

In [23]:
sys.executable

'/opt/conda/envs/opennotes/bin/python'

In [24]:
!/opt/conda/envs/opennotes/bin/python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz
# !/opt/conda/envs/opennotes/bin/python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_md-0.4.0.tar.gz
!/opt/conda/envs/opennotes/bin/python -my pip uninstall negspacy
!/opt/conda/envs/opennotes/bin/python -m pip install spacy==2.3.5

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz (33.1 MB)
/opt/conda/envs/opennotes/bin/python: No module named y


In [25]:
import spacy
spacy.__version__

'2.3.5'

In [26]:
sci_nlp = spacy.load("en_core_sci_sm")
# sci_nlp = spacy.load("en_core_sci_md")

### b) Med7

For Med7, we set up their model that has been trained specifically for NER of medication-related concepts: dosage, drug names, duration, form, frequency, route of administration, and strength. The model is trained on MIMIC-III, so it should work well for us.

In [27]:
spacy.__version__

'2.3.5'

In [28]:
# installs Med7 model
!pip install https://www.dropbox.com/s/xbgsy6tyctvrqz3/en_core_med7_lg.tar.gz?dl=1

     |██████████████████              | 500.5 MB 74.4 MB/s eta 0:00:064

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████▉   | 804.5 MB 9.2 MB/s eta 0:00:1002

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 892.8 MB 8.0 kB/s  eta 0:00:01


In [29]:
med7_nlp = spacy.load("en_core_med7_lg")

Run a quick demo example:

In [30]:
# # create distinct colours for labels
# col_dict = {}
# seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
# for label, colour in zip(med7_nlp.pipe_labels['ner'], seven_colours):
#     col_dict[label] = colour

# options = {'ents': med7_nlp.pipe_labels['ner'], 'colors':col_dict}

# text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
# doc = med7_nlp(text)

# spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

# # [(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
for text in all_sent_strs[20:40]:
    print(text)

    doc = med7_nlp(text)
    print(doc.ents)
    print([(ent.text, ent.label_) for ent in doc.ents])

    spacy.displacy.render(doc, style='ent')
    print("**********************************")

In [76]:
spacy.displacy.render(doc) #, style='ent', jupyter=True)

## iii) Entity Linker

The EntityLinker is a spaCy component that links to a knowledge base. The linker compares words with the concepts in the specified knowledge base (e.g. scispaCy's UMLS does some form of character overlap-based nearest neighbor search, has option to resolve abbreviations first).

[Note: Entities generally get resolved to a list of different entities. This [blog post](http://sujitpal.blogspot.com/2020/08/disambiguating-scispacy-umls-entities.html) describes one potential way to disambiguate this by figuring out "most likely" set of entities. Gonna start off with just resolving to the 1st entity tho... hopefully that's sufficient.]

### a) scispaCy

#### UMLS Linker

UMLS linker maps entities to the UMLS concept. Main parts we'll be interested in are: semantic type and concept (mainly the common name, maybe the CUI might become important later).

* _Semantic type_ is the broader category that the entity falls under, e.g. disease, pharmacologic substance, etc. See [this](https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt) for a full list.

* _Concepts_ refer to the more fundamental entity itself, e.g. pneumothorax, ventillator, etc. Many concepts can fall under a semantic type.

More info on `UmlsEntityLinker` ([source code](https://github.com/allenai/scispacy/blob/4ade4ec897fa48c2ecf3187caa08a949920d126d/scispacy/linking.py#L9))

In [89]:
from scispacy.umls_linking import UmlsEntityLinker
# import scispacy.rxnorm_linking

# abbreviation_pipe = AbbreviationDetector(nlp) # automatically included with UMLS linker
# nlp.add_pipe(abbreviation_pipe)
linker = UmlsEntityLinker(k=10,                          # number of nearest neighbors to look up from
                          threshold=0.7,                 # confidence threshold to be added as candidate
                          max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
                          filter_for_definitions=False,  # no definition is OK
                          resolve_abbreviations=True)    # resolve abbreviations before linking
sci_nlp.add_pipe(linker)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### b) Med7 

No need for entity linker

### c) Negspacy

In [196]:
sys.executable

'/opt/conda/envs/opennotes/bin/python'

In [197]:
# !/opt/conda/envs/opennotes/bin/python -m pip install negspacy

In [198]:
# from negspacy.negation import Negex

# nlp = spacy.load("en_core_web_sm")
# negex = Negex(nlp, ent_types=["PERSON','ORG"])
# nlp.add_pipe(negex, last=True)

In [199]:
spacy.__version__

'2.3.5'

In [200]:
import sys
print(sys.executable)

/opt/conda/envs/opennotes/bin/python


## iv) Putting everything together

### a) Processing a sentence

Here, we'll walk through how we will process and represent a sentence. This will be wrapped up in a `SentenceRep` class.

#### Categorizing type of conflict

The first larger task is to categorize by the type of conflict to check for since our method will likely be different (at least for the rule based). We wrote up a short list [here](https://docs.google.com/document/d/1fEBk0JHeyQWshYWW5w_VTkaYyRfm9MBxJ9DAGoVa8Yw/edit?usp=sharing). 

To do this, we're using the semantic type that is identified by the UMLS linker. Here's a table of the semantic types we're filtering for, and which conflict they'll be used for.

Here's a [full list](https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt) of semantic types. You can look up definitions of semantic types [here](http://linkedlifedata.com/resource/umls-semnetwork/T033).

| Conflict | Semantic Type |
| --- | ----------- |
| Diagnoses-related errors | Disease or Syndrome (T047), Diagnostic Procedure(T060) |
| Inaccurate description of medical history (symptoms) | Sign or Symptom (T184) |
| Inaccurate description of medical history (operations) | Therapeutic or Preventive Procedure (T061) |
| Inaccurate description of medical history (other) | [all of the above and below] |
| Medication or allergies | Clinical Drug (T200), Pharmacologic Substance (T121) |
| Test procedures or results | Laboratory Procedure (T059), Laboratory or Test Result (T034) | 


<!-- Some useful def's 
Finding - 
That which is discovered by direct observation or measurement of an organism attribute or condition, including the clinical history of the patient. The history of the presence of a disease is a 'Finding' and is distinguished from the disease itself.  -->

For clarity, the concepts we'll keep from the UMLS linker are anything falling into these semantic types (which we will then categorize by type of conflict using the table above):

* T047 - Disease or Syndrome
* T121 - Pharmacologic Substance
* T023 - Body Part, Organ, or Organ Component
* T061 - Therapeutic or Preventive Procedure 
* T060 - Diagnostic Procedure
* T059 - Laboratory Procedure
* T034 - Laboratory or Test Result 
* T184 - Sign or Symptom 
* T200 - Clinical Drug

We'll store this info into a dictionary now.

In [77]:
SEMANTIC_TYPES = ['T047', 'T121', 'T023', 'T061', 'T060', 'T059', 'T034', 'T184', 'T200']
SEMANTIC_NAMES = ['Disease or Syndrome', 'Pharmacologic Substance', 'Body Part, Organ, or Organ Component', \
                  'Therapeutic or Preventive Procedure', 'Diagnostic Procedure', 'Laboratory Procedure', \
                  'Laboratory or Test Result', 'Sign or Symptom', 'Clinical Drug']
SEMANTIC_TYPE_TO_NAME = dict(zip(SEMANTIC_TYPES, SEMANTIC_NAMES))

SEMANTIC_TYPE_TO_NAME

{'T047': 'Disease or Syndrome',
 'T121': 'Pharmacologic Substance',
 'T023': 'Body Part, Organ, or Organ Component',
 'T061': 'Therapeutic or Preventive Procedure',
 'T060': 'Diagnostic Procedure',
 'T059': 'Laboratory Procedure',
 'T034': 'Laboratory or Test Result',
 'T184': 'Sign or Symptom',
 'T200': 'Clinical Drug'}

We'll also store the mapping from conflicts to semantic types.

In [78]:
CONFLICT_TO_SEMANTIC_TYPE = {
    "diagnosis": {'T047', 'T060'},
    "med_history_symptom": {'T184'},
    "med_history_operation": {'T061'},
    "med_history_other": set(SEMANTIC_TYPES),
    "med_allergy": {'T200', 'T121'},
    "test_results": {'T059', 'T034'}
}

CONFLICT_TO_SEMANTIC_TYPE

{'diagnosis': {'T047', 'T060'},
 'med_history_symptom': {'T184'},
 'med_history_operation': {'T061'},
 'med_history_other': {'T023',
  'T034',
  'T047',
  'T059',
  'T060',
  'T061',
  'T121',
  'T184',
  'T200'},
 'med_allergy': {'T121', 'T200'},
 'test_results': {'T034', 'T059'}}

#### Any other design choices, e.g. which info are we storing from NER? what do we store from Med7? [TODO]

#### SentenceRep, need to integrate Med7 [TODO]

In [79]:
class SentRep(object):
    def __init__(self, sentence, sci_nlp, linker, med7_nlp, filter_map=None, conflict_map=None):
        """ 
        Given a sentence and scispacy model with UMLS linker and Med7 model,
        SentRep extracts important information and stores them as attributes. 
        
        Set filter_map to be map of semantic type identifiers to name (e.g. SEMANTIC_TYPE_TO_NAME)
        to only extract those attributes.
        """
        self.umls_cui_map = linker.umls.cui_to_entity # maps CUI to entity information
        self.filter_map   = filter_map
        self.conflict_map = conflict_map
        self.is_filter    = (filter_map is not None)
        self.is_conflict  = (conflict_map is not None)
        
        self.sci_doc = sci_nlp(sentence)
        self.med_doc = med7_nlp(sentence)
        
        self.semantic_types = []
        self.semantic_names = []  # names of categories of entities
        self.canonical_names = [] # names of types of entities
        self.get_umls_info()

    def get_umls_info(self):
        for ent in self.sci_doc.ents: # extract info (umls) for each entity
            # todo: look into this bug, ent._.umls_ents sometimes empty list
            try:
                cui, _ = ent._.umls_ents[0] # assuming `max_entites_per_mention=1` for now
            except IndexError:
                continue
            cui_info = self.umls_cui_map[cui]
            
            ent_valid_type_list = [t in self.filter_map for t in cui_info.types]
            ent_valid_type = any(ent_valid_type_list) # checks if entity is a valid type
            
            if not self.is_filter or ent_valid_type: # only add to list if we're not filtering of it's valid
                self.canonical_names.append(cui_info.canonical_name)
                for (stype, keep) in zip(cui_info.types, ent_valid_type_list):
                    if keep:
                        self.semantic_types.append(stype)
                        self.semantic_names.append(self.filter_map[stype])
            
        self.semantic_types = set(self.semantic_types)
        self.semantic_names = set(self.semantic_names)
        self.canonical_names = set(self.canonical_names)
        
    def similarity(self, srep):
        """ Given another SentenceRep instance, compares similarity.
        
        e.g.
        srep.similarity(srep)   # measuring similarity with itself
        >> 1.0                  # 1.0 is maximum score
        """
        return self.sci_doc.similarity(srep.doc)
    
    def is_ctype(self, ctype):
        """ Given a conflict type (e.g. "diagnosis"),
            returns True if this sentence falls into that category, False otherwise.
            Returns None if conflict_map is undefined.
        """
        if self.is_conflict: 
            ctype_stypes = self.conflict_map[ctype] # get list of semantic types for this conflict
            return any([stype in ctype_stypes for stype in self.semantic_types])
        return None

Let's run this on an example.

In [80]:
sents_dict.keys()

dict_keys([556, 571617, 571618, 571622, 571742, 571932, 571731, 571814, 571689, 572002, 571629, 571849, 571968, 1076941, 1077531, 1076906, 1077579, 1076756, 1076739])

In [81]:
note_category = list(map(lambda x: x[2], sents_dict.values()))
note_category_dict = dict(zip(sents_dict.keys(), note_category))

note_category_dict

{556: 'Discharge summary',
 571617: 'Nursing',
 571618: 'Physician ',
 571622: 'Physician ',
 571742: 'Respiratory ',
 571932: 'Physician ',
 571731: 'Physician ',
 571814: 'Nursing',
 571689: 'Physician ',
 572002: 'Nursing',
 571629: 'Nursing',
 571849: 'Nursing',
 571968: 'Physician ',
 1076941: 'Radiology',
 1077531: 'Radiology',
 1076906: 'Radiology',
 1077579: 'Radiology',
 1076756: 'Radiology',
 1076739: 'Radiology'}

### b) Processing a note

Now that we have a way to process sentences, how can we process an entire note (or set of notes) and figure out how to flag for potential conflicts?

Let's start off by processing all the sentences in the notes we've loaded up.

In [82]:
note_category_dict

{556: 'Discharge summary',
 571617: 'Nursing',
 571618: 'Physician ',
 571622: 'Physician ',
 571742: 'Respiratory ',
 571932: 'Physician ',
 571731: 'Physician ',
 571814: 'Nursing',
 571689: 'Physician ',
 572002: 'Nursing',
 571629: 'Nursing',
 571849: 'Nursing',
 571968: 'Physician ',
 1076941: 'Radiology',
 1077531: 'Radiology',
 1076906: 'Radiology',
 1077579: 'Radiology',
 1076756: 'Radiology',
 1076739: 'Radiology'}

In [83]:
# all_sent_strs = sents_dict[471201][0]  # this has been processed above to includ all physician notes

In [90]:
srep = SentRep(sent, sci_nlp, linker, med7_nlp,
                       filter_map=SEMANTIC_TYPE_TO_NAME,
                       conflict_map=CONFLICT_TO_SEMANTIC_TYPE)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [92]:
srep.med_doc

Has remained afebrile    overnight with improvement in her WBC.'

In [84]:
all_sreps = []
all_srep_canon_names = []
all_srep_times = []
for i, sent in tqdm(enumerate(all_sent_strs)):
    try: # some sentences run into errors it seems.. todo: look into
        srep = SentRep(sent, sci_nlp, linker,
                       filter_map=SEMANTIC_TYPE_TO_NAME,
                       conflict_map=CONFLICT_TO_SEMANTIC_TYPE)
        if len(srep.canonical_names) > 0:
            all_sreps.append(srep)
            all_srep_canon_names.append(list(srep.canonical_names))
            all_srep_times.append(all_sent_times[i])
    except:
        continue

280it [00:00, 586615.94it/s]


In [85]:
len(all_sreps)

0

In [420]:
# Some concepts we've identified
corpus = list(map(lambda x: ' '.join(x), all_srep_canon_names))
corpus[:5]

['acute stress Nose Ear structure',
 'Percussion Left radial vein',
 'Blood product',
 'Diverticulitis',
 'Lithotripsy']

#### Filter by conflict type, filter by semantic type, group by similar concept, then look for negations/disease-symptom mismatch/prescription incorrect info

Comments:

* Working pretty okay up to grouping sentences falling under 1) same conflict type, and 2) similar concept, e.g. ventricular cavity size.

* Need to work on the latter part of looking for negations/disease-symptom mismatch/prescription incorrect info

* Some notes -- not all concepts get picked up (inevitable), some abbreviations are resolved incorrectly or not at all, incorrect concept

In [ ]:
CONFLICT_TO_SEMANTIC_TYPE.keys()

README: You can comment out the conflict type of interest here!

In [ ]:
# conflict_type = "diagnosis" # set a conflict category
# conflict_type = "med_history_symptom" # set a conflict category
# conflict_type = "med_history_operation" # set a conflict category
# conflict_type = "med_history_other" # set a conflict category
# conflict_type = "med_allergy" # set a conflict category
conflict_type = "test_results" # set a conflict category

# filter sreps & corresponding datetimes by conflict type
all_sreps_and_times = list(zip(all_sreps, all_srep_times))
conflict_sreps_and_times = list(filter(lambda x: x[0].is_ctype(conflict_type), all_sreps_and_times))
conflict_sreps, conflict_sreps_times = list(zip(*conflict_sreps_and_times)) # unzip

print(f"We have {len(conflict_sreps)} sentences that are {conflict_type} related.")

In [ ]:
len(all_sreps)

In [ ]:
SEMANTIC_TYPE_TO_NAME

In [ ]:
CONFLICT_TO_SEMANTIC_TYPE[conflict_type]

README: You can set `semantic_type` to be any of the semantic types printed in the cell above (any semantic type for the conflict type we selected above).

In [ ]:
# filter sreps by semantic type
semantic_type = "T034"
semantic_sreps = list(filter(lambda x: semantic_type in x.semantic_types, conflict_sreps))

# filter sreps & corresponding datetimes by conflict type
all_conflict_sreps_and_times = list(zip(conflict_sreps, conflict_sreps_times))
semantic_sreps_and_times = list(filter(lambda x: semantic_type in x[0].semantic_types, \
                                       all_conflict_sreps_and_times))
semantic_sreps, semantic_sreps_times = list(zip(*semantic_sreps_and_times)) # unzip

print(f"We have {len(semantic_sreps)} sentences that are \"{SEMANTIC_TYPE_TO_NAME[semantic_type]}\" related.")

In [ ]:
# get canonical names for these sentences
semantic_sreps_canon_names = list(map(lambda x: x.canonical_names, semantic_sreps))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
corpus = list(map(lambda x: ' '.join(x), semantic_sreps_canon_names))
X = vectorizer.fit_transform(corpus)
X = X.toarray()

sim_X = cosine_similarity(X) # larger=more similar
simx, simy = np.where(sim_X>0.5)

README: This prints out similar sentences, time of note, and extracted concepts. You can look through these sentences and analyze which ones might indicate potential conflict and past them here to send to Dr. Saenz. https://docs.google.com/document/u/5/d/1Q2mxfOd3Wwiqp_rBQVEYTUPHBfeGGqSBx9acQ88fGOA/edit?usp=sharing

In [ ]:
semantic_sreps_times[11].date() == semantic_sreps_times[19].date()

In [ ]:
## Same day only
# conflict_type = "symptom" # set a conflict category
# conflict_type = "test_results" # set a conflict category

## Any day
# conflict_type = "diagnosis" # set a conflict category
# conflict_type = "med_history_symptom" # set a conflict category
# conflict_type = "med_history_operation" # set a conflict category
# conflict_type = "med_history_other" # set a conflict category
# conflict_type = "med_allergy" # set a conflict category

In [ ]:
dynamic_conflict_types = {"symptom", "test_results"}
static_conflict_types  = {"diagnosis", "med_history_symptom", "med_history_operation", \
                          "med_history_other", "med_allergy"}
is_nonzero_time = lambda time: time.hour != 0 or time.minute != 0
def is_comparable_time(dt1, dt2, conflict_type):
    # if time is non-zero and datetime is exact same, it's a copy, so return False
    if (is_nonzero_time(dt1) and is_nonzero_time(dt2)) and dt1 == dt2:
        return False
    elif conflict_type in static_conflict_types:
        return True
    return dt1.date() == dt2.date()

In [ ]:
int(hadm_id)

In [ ]:
processed_dir = 'processed'
os.makedirs(processed_dir, exist_ok=True)

pt_csv = os.path.join(processed_dir, f"{int(hadm_id)}.csv")

In [ ]:
df.to_csv(pt_csv)

In [ ]:
processed_pairs = []
for x, y in zip(simx, simy):
    # removing same sentence pairs, checking dates
    if x>y and is_comparable_time(semantic_sreps_times[x], semantic_sreps_times[y], conflict_type):
        print(f"Cosine similarity: {sim_X[x, y]}")
        print("----- SENTENCE 1 -----")
        print(f">> Time: {semantic_sreps_times[x]}\n" +\
              f">> Concepts: {semantic_sreps_canon_names[x]}\n" +\
              f">> {semantic_sreps[x].doc}")
        print("----- SENTENCE 2 -----")
        print(f">> Time: {semantic_sreps_times[y]}\n" +\
              f">> Concepts: {semantic_sreps_canon_names[y]}\n" +\
              f">> {semantic_sreps[y].doc}")
        print("**********************************")
        
        # save
        processed_pairs.append([semantic_sreps[x].doc, semantic_sreps[y].doc, \
                                semantic_sreps_times[x], semantic_sreps_times[y], \
                                semantic_sreps_canon_names[x], semantic_sreps_canon_names[y], \
                                sim_X[x, y], SEMANTIC_TYPE_TO_NAME[semantic_type]])
        
df = \
pd.DataFrame(np.array(processed_pairs), \
             columns=["sentence 1", "sentence 2", \
                      "time 1", "time 2", \
                      "concepts 1", "concepts 2", \
                      "cosine similarity", "semantic type"])

## AUTOMATED

In [421]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Do dynamic for now if unclear
# dynamic_semantic_types = {"T121", "T023", "T059", "T034", "T184", "T200"}
# static_semantic_types  = {"T047", "T061", "T060"}
dynamic_semantic_types = {"T121", "T023", "T059", "T034", "T184", "T200", "T047", "T061", "T060"}
static_semantic_types  = {}
is_nonzero_time = lambda time: time.hour != 0 or time.minute != 0
def is_comparable_time(dt1, dt2, semantic_type):
#     # if time is non-zero and datetime is exact same, it's a copy, so return False
#     if (is_nonzero_time(dt1) and is_nonzero_time(dt2)) and dt1 == dt2:
#         return False
    if semantic_type in static_semantic_types:
        return True
    return dt1.date() == dt2.date()

In [422]:
processed_dir = 'processed'
os.makedirs(processed_dir, exist_ok=True)

pt_csv = os.path.join(processed_dir, f"{int(hadm_id)}.csv")

processed_pairs = []

In [423]:
for semantic_type in SEMANTIC_TYPE_TO_NAME:
    # filter sreps by semantic type
    # semantic_type = "T034"
    semantic_sreps = list(filter(lambda x: semantic_type in x.semantic_types, all_sreps))

    # filter sreps & corresponding datetimes by conflict type
    all_conflict_sreps_and_times = list(zip(all_sreps, all_srep_times))
    semantic_sreps_and_times = list(filter(lambda x: semantic_type in x[0].semantic_types, \
                                           all_conflict_sreps_and_times))
    
    if len(semantic_sreps_and_times) == 0:
        continue
    semantic_sreps, semantic_sreps_times = list(zip(*semantic_sreps_and_times)) # unzip

    print(f"We have {len(semantic_sreps)} sentences that are \"{SEMANTIC_TYPE_TO_NAME[semantic_type]}\" related.")

    # get canonical names for these sentences
    semantic_sreps_canon_names = list(map(lambda x: x.canonical_names, semantic_sreps))

    # get cosine sim
    vectorizer = CountVectorizer()
    corpus = list(map(lambda x: ' '.join(x), semantic_sreps_canon_names))
    X = vectorizer.fit_transform(corpus)
    X = X.toarray()

    sim_X = cosine_similarity(X) # larger=more similar
    simx, simy = np.where(sim_X>0.5)

    for x, y in zip(simx, simy):
        # removing same sentence pairs, checking dates
        if x>y and is_comparable_time(semantic_sreps_times[x], semantic_sreps_times[y], semantic_type):
            print(f"Cosine similarity: {sim_X[x, y]}")
            print("----- SENTENCE 1 -----")
            print(f">> Time: {semantic_sreps_times[x]}\n" +\
                  f">> Concepts: {semantic_sreps_canon_names[x]}\n" +\
                  f">> {semantic_sreps[x].doc}")
            print("----- SENTENCE 2 -----")
            print(f">> Time: {semantic_sreps_times[y]}\n" +\
                  f">> Concepts: {semantic_sreps_canon_names[y]}\n" +\
                  f">> {semantic_sreps[y].doc}")
            print("**********************************")

            # save
            processed_pairs.append([semantic_sreps[x].doc, semantic_sreps[y].doc, \
                                    semantic_sreps_times[x], semantic_sreps_times[y], \
                                    semantic_sreps_canon_names[x], semantic_sreps_canon_names[y], \
                                    sim_X[x, y], SEMANTIC_TYPE_TO_NAME[semantic_type]])
        
        
df = \
pd.DataFrame(np.array(processed_pairs), \
             columns=["sentence 1", "sentence 2", \
                      "time 1", "time 2", \
                      "concepts 1", "concepts 2", \
                      "cosine similarity", "semantic type"])

df.to_csv(pt_csv)

We have 67 sentences that are "Disease or Syndrome" related.
Cosine similarity: 0.7071067811865475
----- SENTENCE 1 -----
>> Time: 2157-05-16 20:06:00
>> Concepts: {'Sepsis', 'Diverticulitis'}
>> Post operatively patient with significant fever and tachycardia    concerning for sepsis with both urinary source and possible    diverticulitis contributing.'
----- SENTENCE 2 -----
>> Time: 2157-05-16 20:06:00
>> Concepts: {'Diverticulitis'}
>> This is in the setting of recent Rx    for diverticulitis and patient reasonably unremarkable prior to    procedure.'
**********************************
Cosine similarity: 1.0
----- SENTENCE 1 -----
>> Time: 2157-05-16 20:06:00
>> Concepts: {'Diverticulitis'}
>> Patient at that time had ABX and conservative Rx    for diverticulitis.'
----- SENTENCE 2 -----
>> Time: 2157-05-16 20:06:00
>> Concepts: {'Diverticulitis'}
>> This is in the setting of recent Rx    for diverticulitis and patient reasonably unremarkable prior to    procedure.'
****************

/opt/conda/envs/opennotes/lib/python3.7/site-packages/ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


README: This prints out similar sentences, time of note, and extracted concepts. You can look through these sentences and analyze which ones might indicate potential conflict and past them here to send to Dr. Saenz. https://docs.google.com/document/u/5/d/1Q2mxfOd3Wwiqp_rBQVEYTUPHBfeGGqSBx9acQ88fGOA/edit?usp=sharing

In [75]:
## Same day only
# conflict_type = "symptom" # set a conflict category
# conflict_type = "test_results" # set a conflict category

## Any day
# conflict_type = "diagnosis" # set a conflict category
# conflict_type = "med_history_symptom" # set a conflict category
# conflict_type = "med_history_operation" # set a conflict category
# conflict_type = "med_history_other" # set a conflict category
# conflict_type = "med_allergy" # set a conflict category

# [IGNORE FOR NOW]

### Experiment on MIMIC

More info on `UmlsEntityLinker` ([source code](https://github.com/allenai/scispacy/blob/4ade4ec897fa48c2ecf3187caa08a949920d126d/scispacy/linking.py#L9))

`nlp` is a `Language` (?) object that returns a `Doc` ([source code](https://github.com/explosion/spaCy/blob/master/spacy/tokens/doc.pyx#L104)), which is a sequence of `Token` word objects ([source code](https://github.com/explosion/spaCy/blob/master/spacy/tokens/token.pyx#L26)).

https://spacy.io/api/doc

https://spacy.io/api/token

#### Default vectorization of sentences and words

We start off by choosing a model [here](https://allenai.github.io/scispacy/), e.g. `en_core_sci_sm`. The model we choose is tuned for biomedical vocabulary and has a large collection of word vectors good for our vocabulary. 

When we pass a sentence through the model, e.g. `nlp`, each word in the sentence gets vectorized. These word vecs can be accessed through the output `Doc` instance. This instance also contains sentence vecs, which are just averaged across the word vecs. 

#### From word vectorization to entity recognition

Once words are vectorized, scispacy runs KNN in the background to map words to the entity labels specified by our linkers. 

#### Measuring similarity 

We can leverage the vectorizations to measure similarities. For example, we can use the function `Doc.similarity` ([source code](https://spacy.io/api/doc#similarity)) to measure similarity between different sentences. Their default uses cosine similarity using average of word vectors as their sentence vectorization. 

Note that we can also measure similarity between other instances (e.g. `Token`), or between two different instances (e.g. between `Token` and `Doc`). 

In [115]:
nlp = spacy.load("en_core_sci_sm")

In [174]:
from scispacy.umls_linking import UmlsEntityLinker

In [175]:
# abbreviation_pipe = AbbreviationDetector(nlp) # automatically included with UMLS linker
# nlp.add_pipe(abbreviation_pipe)
linker = UmlsEntityLinker(k=10,                          # number of nearest neighbors to look up from
                          threshold=0.7,                 # confidence threshold to be added as candidate
                          max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
                          filter_for_definitions=False,  # no definition is OK
                          resolve_abbreviations=True)    # resolve abbreviations before linking
# linker = EntityLinker(k=10,                          # number of nearest neighbors to look up from
#                       threshold=0.7,                 # confidence threshold to be added as candidate
#                       max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
#                       resolve_abbreviations=True,    # resolve abbreviations before linking
#                       linker_name='umls')
nlp.add_pipe(linker)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [224]:
SEMANTIC_TYPE_TO_NAME

{'T047': 'Disease or Syndrome',
 'T121': 'Pharmacologic Substance',
 'T023': 'Body Part, Organ, or Organ Component',
 'T061': 'Therapeutic or Preventive Procedure',
 'T060': 'Diagnostic Procedure',
 'T059': 'Laboratory Procedure'}

In [176]:
class SentRep(object):
    def __init__(self, sentence, nlp, linker, filter_map=None):
        """ 
        Given a sentence and scispacy model with UMLS linker, 
        SentRep extracts important information and stores them as attributes. 
        
        Set filter_map to be map of semantic type identifiers to name (e.g. SEMANTIC_TYPE_TO_NAME)
        to only extract those attributes.
        """
        self.umls_cui_map = linker.umls.cui_to_entity # maps CUI to entity information
        self.filter_map   = filter_map
        self.is_filter    = (filter_map is not None)
        
        self.doc = nlp(sentence)
        
        self.semantic_types = []
        self.semantic_names = []  # names of categories of entities
        self.canonical_names = [] # names of types of entities
        self.get_umls_info()

    def get_umls_info(self):
        for ent in self.doc.ents: # extract info (umls) for each entity
            cui, _ = ent._.umls_ents[0] # assuming `max_entites_per_mention=1` for now
            cui_info = self.umls_cui_map[cui]
            
            ent_valid_type_list = [t in self.filter_map for t in cui_info.types]
            ent_valid_type = any(ent_valid_type_list) # checks if entity is a valid type
            
            if not self.is_filter or ent_valid_type: # only add to list if we're not filtering of it's valid
                self.canonical_names.append(cui_info.canonical_name)
                for (stype, keep) in zip(cui_info.types, ent_valid_type_list):
                    if keep:
                        self.semantic_types.append(stype)
                        self.semantic_names.append(self.filter_map[stype])
            
        self.semantic_types = set(self.semantic_types)
        self.semantic_names = set(self.semantic_names)
        self.canonical_names = set(self.canonical_names)

In [177]:
srep = SentRep(sent, nlp, linker, filter_map=SEMANTIC_TYPE_TO_NAME)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [178]:
srep.semantic_types, srep.semantic_names, srep.canonical_names

({'T121'}, {'Pharmacologic Substance'}, {'Solu-Medrol'})

In [179]:
SEMANTIC_TYPE_TO_NAME

{'T047': 'Disease or Syndrome',
 'T121': 'Pharmacologic Substance',
 'T023': 'Body Part, Organ, or Organ Component',
 'T061': 'Therapeutic or Preventive Procedure',
 'T060': 'Diagnostic Procedure',
 'T059': 'Laboratory Procedure'}

In [182]:
len(all_sent_strs)

1137

In [201]:
all_sreps = []
all_srep_canon_names = []
for sent in all_sent_strs:
    try:
        srep = SentRep(sent, nlp, linker, filter_map=SEMANTIC_TYPE_TO_NAME)
        if len(srep.canonical_names) > 0:
            all_sreps.append(srep)
            all_srep_canon_names.append(list(srep.canonical_names))
    except:
        continue

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [202]:
len(all_sreps)

293

In [210]:
corpus = list(map(lambda x: ' '.join(x), all_srep_canon_names))
corpus[:10]

['amlodipine',
 'Operative Surgical Procedures Invasive procedure',
 'Esophagogastroduodenoscopy Intubation',
 'oxygen COPD pharmacologic substance Congestive heart failure',
 'COPD pharmacologic substance',
 'Levaquin',
 'Lung consolidation Plain chest X-ray',
 'Electrocardiography',
 'Leukocytosis Lactic acid measurement',
 'Intubation']

In [211]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
corpus = list(map(lambda x: ' '.join(x), all_srep_canon_names))
X = vectorizer.fit_transform(corpus)
X = X.toarray()

In [214]:
sim_X = cosine_similarity(X)
sim_X

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [216]:
(sim_X>0).sum()

3571

In [218]:
simx, simy = np.where(sim_X>0)

In [223]:
for x, y in zip(simx, simy):
    if x!=y:
        print(f"S1: {all_sreps[x].doc}")
        print(f"S2: {all_sreps[y].doc}")
        print(f"S1 canonical concepts: {all_srep_canon_names[x]}")
        print(f"S2 canonical concepts: {all_srep_canon_names[y]}")
        print("**********************************")

S1: Allergies: Amlodipine
S2: Allergies: Amlodipine
S1 canonical concepts: ['amlodipine']
S2 canonical concepts: ['amlodipine']
**********************************
S1: Major Surgical or Invasive Procedure:
S2: [**Last Name (STitle) 1407**]  was very involved in discussions regarding patient's code status.
S1 canonical concepts: ['Operative Surgical Procedures', 'Invasive procedure']
S2 canonical concepts: ['Discussion (procedure)']
**********************************
S1: Major Surgical or Invasive Procedure:
S2: The patient's family felt that she would not want a tracheostomy.
S1 canonical concepts: ['Operative Surgical Procedures', 'Invasive procedure']
S2 canonical concepts: ['Tracheostomy procedure']
**********************************
S1: Major Surgical or Invasive Procedure:
S2: A tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach.
S1 canonical concepts: ['Operative Surgical Procedures', 'Invasive procedure']
S2 canonical

#### What to do from here

* group sentences by semantic type and map them to each category of conflicts to check for (e.g. T047 --> disease and symptom errors)
* use canonical names to group sentences with similar or dissimilar topics together (note: once we incorporate RxNorm and Med7, include concepts from those into canonical names as well)

In [22]:
doc = nlp(note_str) 
# doc = nlp(abbreviation_note_str)
displacy_image = displacy.render(doc, jupyter=True, style='ent')

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [70]:
entity = doc.ents
entity = [str(item) for item in entity]               # convert each entity tuple to list of strings
entity = str(OrderedDict.fromkeys(entity))            # returns unique entities only
entity = nlp(entity).ents                             # convert unique entities back to '.ents' object

In [71]:
# Extracting info from "ents" object
ent = entity[1]

# 1. Name of entity
print(ent)

# 2. UMLS mapping
umls_ents = ent._.umls_ents
print(umls_ents)

# 3. CUI, confidence
umls_ent = umls_ents[0]
cui, confidence = umls_ent
print(f"CUI: {cui}, Confidence: {confidence}")

# 4. CUI information
ent_cui_info = linker.umls.cui_to_entity[cui]
print(ent_cui_info)

bulbar muscular atrophy
[('C1839259', 0.9103652834892273), ('C0026846', 0.800467312335968)]
CUI: C1839259, Confidence: 0.9103652834892273
CUI: C1839259, Name: Bulbo-Spinal Atrophy, X-Linked
Definition: An X-linked recessive form of spinal muscular atrophy. It is due to a mutation of the gene encoding the ANDROGEN RECEPTOR.
TUI(s): T047
Aliases (abbreviated, total: 44): 
	 Bulbo-Spinal Atrophy, X-Linked, Atrophies, X-Linked Bulbo-Spinal, Bulbo Spinal Atrophy, X Linked, Bulbo-Spinal Atrophies, X-Linked, X-Linked Bulbo-Spinal Atrophies, Atrophy, X-Linked Bulbo-Spinal, X Linked Bulbo Spinal Atrophy, X-Linked Bulbo-Spinal Atrophy, Bulbospinal muscular atrophy, Kennedy Syndrome


In [79]:
print(linker.umls.cui_to_entity[umls_ents[0][0]].canonical_name)
print(linker.umls.cui_to_entity[umls_ents[1][0]].canonical_name)

Bulbo-Spinal Atrophy, X-Linked
Muscular Atrophy


#### Accessing info

In [80]:
# ent_cui_info.aliases
# ent_cui_info.canonical_name
# ent_cui_info.concept_id
# ent_cui_info.count
# ent_cui_info.definition
# ent_cui_info.index
ent_cui_info.types # https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/PSY/stats.html

['T047']

https://medium.com/@oyewusiwuraola/how-to-use-scispacy-for-biomedical-named-entity-recognition-abbreviation-resolution-and-link-umls-87d3f7c08db2

# Other: Relevant papers

## [MedNLI](https://www.aclweb.org/anthology/D18-1187.pdf)



In [ ]:
!wget -r -N -c -np --user yutsumi --ask-password https://physionet.org/files/mednli/1.0.0/

In [ ]:
!git clone https://github.com/jgc128/mednli.git

In [ ]:
!pip install -r mednli/requirements.txt

In [ ]:
!pip install sacred

     |████████████████████████████████| 112kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 3.8MB/s 
     |████████████████████████████████| 163kB 7.6MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=fa0ca522388f4c7e3a73ff2256acdbbac77381ab6c0b9fa307a24d2c504fcb6e
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
Successfully built py-cpuinfo


In [ ]:
!mkdir mednli/data

mkdir: cannot create directory ‘mednli/data’: No such file or directory


In [ ]:
!mv physionet.org/files/mednli/1.0.0 mednli/data

mv: cannot stat 'physionet.org/files/mednli/1.0.0': No such file or directory


In [ ]:
import json
import pandas as pd

In [ ]:
filename = 'mednli/data/1.0.0/mli_dev_v1.jsonl'

all_rows = []
with open(str(filename)) as f:
    for i, line in enumerate(f):
        row = json.loads(line)
        if row['gold_label'] != '-':
            all_rows.append(row)

nli_data = pd.DataFrame(all_rows)

FileNotFoundError: ignored

In [ ]:
nli_sub_data = nli_data[['sentence1', 'sentence2', 'gold_label']]

nli_sub_data.loc[nli_sub_data.gold_label == 'contradiction']

In [ ]:
print(f"*****************************")
for _, row in nli_sub_data.loc[nli_sub_data.gold_label == 'contradiction'].iterrows():
  print(f"Sentence 1: {row.sentence1}")
  print(f"Sentence 2: {row.sentence2}")
  print(f"*****************************")